# SETUP

In [ ]:
!nvcc --version
!nvidia-smi

In [ ]:
!python -m pip install paddlepaddle-gpu==2.4.1.post117 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!pip install decord

In [13]:
import sys
import os
HOME = os.getcwd()

In [ ]:
%%bash
git clone https://github.com/PaddlePaddle/PaddleDetection.git
cd PaddleDetection
pip install -r requirements.txt
python setup.py install
cd ../
sys.path.append("PaddleDetection")

In [ ]:
!pip install decord

In [ ]:
!git clone https://github.com/ifzhang/ByteTrack.git
!cd ByteTrack && pip3 install -q -r requirements.txt
!cd ByteTrack && python3 setup.py -q develop
sys.path.append(f"{HOME}/ByteTrack")

In [ ]:
!pip3 install cython
!pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip3 install cython_bbox

In [ ]:
!git clone https://github.com/Wiqzard/hack-sordi-tracking.git
!cd hack-sordi-tracking && git pull
sys.path.append(f"{HOME}/hack-sordi-tracking")

In [ ]:
!yes | pip uninstall opencv-python
!pip install opencv-python-headless

In [ ]:
!python3 -m pip install numpy==1.22.2

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
!pip install Cython
import os
import numpy
import sys

#%cd bbox
#!python3 setup.py build_ext --inplace
#import pyximport
#pyximport.install() 
#pyximport.install(setup_args={"script_args" : ["--verbose"]})
#sys.path.append(f"{HOME}/bbox")
#from bbox import bbox_overlaps
from IPython import display
#display.clear_output()
#%cd ../
#os.getcwd()
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
!cd ByteTrack && pip3 install -q -r requirements.txt
!cd ByteTrack && python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric

display.clear_output()
import sys
sys.path.append(f"{HOME}/ByteTrack")
import yolox
print("yolox.__version__:", yolox.__version__)
print(numpy.version.version)
sys.path.append(f"{HOME}/ByteTrack")
sys.path.append("/home/5qx9nf8a/.local/bin")
display.clear_output()
!git clone https://github.com/Wiqzard/hack-sordi-tracking.git
#!cd tracking-tools && git pull
sys.path.append(f"{HOME}/tracking-tools")

### First transform dataset from bmw format to yolov5. Then from yolov5 to COCO.

# Eval

In [ ]:
%%bash
cd PaddleDetection

WEIGHTS=/home/5qx9nf8a/team_workspace/PaddleDetection/tracking/model_final.pdparams
#configs=/home/5qx9nf8a/team_workspace/PaddleDetection/configs/mot/bytetrack/detector/ppyoloe_plus_m_bytetrack.yml
CONFIGS=/home/5qx9nf8a/team_workspace/PaddleDetection/configs/ppyoloe/ppyoloe_plus_crn_m_80e_coco.yml 

CUDA_VISIBLE_DEVICES=0 python tools/eval.py -c ${CONFIGS} -o weights=${WEIGHTS} --amp


# Export

In [ ]:
%%bash
cd PaddleDetection

CONFIGS=/home/5qx9nf8a/team_workspace/PaddleDetection/configs/ppyoloe/ppyoloe_plus_crn_m_80e_coco.yml 
WEIGHTS=/home/5qx9nf8a/team_workspace/PaddleDetection/tracking/model_final.pdparams

python tools/export_model.py -c ${CONFIGS} -o weights=${WEIGHTS} trt=True

# Test TRT Inference

In [2]:
%%bash
cd PaddleDetection

MODEL_DIR=/home/5qx9nf8a/team_workspace/PaddleDetection/output_inference/ppyoloe_plus_crn_m_80e_coco
VIDEO_PATH=/home/5qx9nf8a/team_workspace/data/Hackathon_Stage2/Evaluation_set/video/eval_video_1.mp4 #--video_file=${VIDEO_PATH}
IMAGE_PATH=/home/5qx9nf8a/team_workspace/data/Hackathon_Stage2/Evaluation_set/dataset/images/124.jpg

CUDA_VISIBLE_DEVICES=0 python deploy/python/infer.py --model_dir=${MODEL_DIR} --image_file=${IMAGE_PATH} --run_mode=trt_fp16 --device=gpu --run_benchmark=True

Error while terminating subprocess (pid=102338): 


# Video Processing (Detection + Tracking + Scanning + Annotating)

In [2]:
from dataclasses import dataclass, field
from __future__ import annotations


@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.2 #0.4
    track_buffer: int = 30
    match_thresh: float = 0.7 #0.7
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

@dataclass(frozen=True)
class Args:
    BYTE_TRACKER_ARGS = BYTETrackerArgs
    STRIDE: int = 8
    REDUCTION_FACTOR: int = 3
    SOURCE_VIDEO_PATH: str = (
        "data/Hackathon_Stage2/Evaluation_set/video/eval_video_1.mp4"
    )
    TARGET_VIDEO_PATH: str = "/temp/eval_video_1.mp4"
    MODEL_DIR: str = "/home/5qx9nf8a/team_workspace/PaddleDetection/output_inference/ppyoloe_plus_crn_m_80e_coco"
    BATCH_SIZE: int = 32
    RUN_MODE: str = "paddle" #"trt_fp16" "paddle", "trt_int8"    trt modes seem to be lagging at first. look for PaddleDetection docs.
    CPU_THREADS: int = 1
    MAX_DETECTIONS: int = 300 # do not change this/ not implemented
    CLASS_NAMES_DICT: dict = field(
        default_factory=lambda:{
            0: "klt_box_empty",
            1: "klt_box_full",
            2: "rack_1",
            3: "rack_2",
            4: "rack_3",
            5: "rack_4",
            6: "placeholder",
        }
    )

    SCANNER_X: int = 300
    SCANNER_Y: int = 50
    BOX_THRESHOLD: float = 0.35
    RACK_THRESHOLD: float = 0.79

args = Args()

In [ ]:
from video_processor.video_processor import VideoProcessor
from PaddleDetection.deploy.python.infer import Detector
import yolox

if __name__ == "__main__":  
    SOURCE_VIDEO_PATH = "data/live_demo/Evaluation_set/demo_eval_video/full_eval_demo_video.mp4"#"data/Hackathon_Stage2/Evaluation_set/video/eval_video_1.mp4"
    TARGET_VIDEO_PATH = "/home/5qx9nf8a/team_workspace/temp/"
    detector = Detector(model_dir=args.MODEL_DIR,
                 device='GPU',
                 run_mode=args.RUN_MODE,
                 batch_size=args.BATCH_SIZE,
                 trt_min_shape=1,
                 trt_max_shape=1280,
                 trt_opt_shape=640,
                 trt_calib_mode=False,
                 cpu_threads=args.CPU_THREADS,
                 enable_mkldnn=False,
                 enable_mkldnn_bfloat16=False,
                 output_dir='output_paddle',
                 threshold=0.3,
                 delete_shuffle_pass=False)
    
    video_processor = VideoProcessor(source_video_path = SOURCE_VIDEO_PATH, 
                                    target_dir = TARGET_VIDEO_PATH,
                                    args = args)
    video_processor.process_video(detector, with_scanner=True, with_placeholders=True, with_annotate_scanner=True)

In [12]:
video_processor.create_submission(mAP=93.3, fps=103.68, save=True)

{'eval_video': [{'rack_name': 'rack_1',
   'rack_conf': 0.983929,
   'N_full_KLT': 14,
   'N_empty_KLT': 11,
   'N_Pholders': 23,
   'shelf_N_Pholders': {'shelf_1': 6,
    'shelf_2': 8,
    'shelf_3': 1,
    'shelf_4': 8}},
  {'rack_name': 'rack_2',
   'rack_conf': 0.961904,
   'N_full_KLT': 8,
   'N_empty_KLT': 7,
   'N_Pholders': 24,
   'shelf_N_Pholders': {'shelf_1': 6,
    'shelf_2': 4,
    'shelf_3': 9,
    'shelf_4': 5}},
  {'rack_name': 'rack_4',
   'rack_conf': 0.984042,
   'N_full_KLT': 6,
   'N_empty_KLT': 7,
   'N_Pholders': 3,
   'shelf_N_Pholders': {'shelf_1': 1, 'shelf_3': 1, 'shelf_4': 1}},
  {'rack_name': 'rack_3',
   'rack_conf': 0.984296,
   'N_full_KLT': 2,
   'N_empty_KLT': 4,
   'N_Pholders': 0,
   'shelf_N_Pholders': {}},
  {'rack_name': 'rack_3',
   'rack_conf': 0.984311,
   'N_full_KLT': 2,
   'N_empty_KLT': 2,
   'N_Pholders': 2,
   'shelf_N_Pholders': {'shelf_2': 1, 'shelf_3': 1}},
  {'rack_name': 'rack_4',
   'rack_conf': 0.985188,
   'N_full_KLT': 5,
   'N_e

In [ ]:
%%bash
cd PaddleDetection/PaddleYOLO
pwd

model_name=ppyoloe 
job_name=ppyoloe_plus_crn_m_80e_coco 

config=/home/5qx9nf8a/team_workspace/PaddleDetection/PaddleYOLO/configs/custom/ppyoloe_plus_crn_m_80e_coco.yml #/configs/custom/${job_name}.yml
log_dir=log_dir/${job_name}
weights=output/${job_name}/model_final.pdparams
video_path=/home/5qx9nf8a/team_workspace/data/Hackathon_Stage2/Evaluation_set/video/eval_video_1.mp4
img_folder=/home/5qx9nf8a/team_workspace/data/trans_data/val/images/
# 1.training（single GPU / multi GPU）
# CUDA_VISIBLE_DEVICES=0 python tools/train.py -c ${config} --eval --amp

# 2.eval
#CUDA_VISIBLE_DEVICES=0 python tools/eval.py -c ${config} -o weights=${weights} --classwise

# 3.infer
#CUDA_VISIBLE_DEVICES=0 python tools/infer.py -c ${config} -o weights=${weights} --infer_dir=${img_folder} --draw_threshold=0.5


# 4.export
CUDA_VISIBLE_DEVICES=0 python tools/export_model.py -c ${config} -o weights=${weights} # trt=True

# CUDA_VISIBLE_DEVICES=0 python tools/export_model.py -c ${config} -o weights=${weights} exclude_post_process=True # trt=True

# CUDA_VISIBLE_DEVICES=0 python tools/export_model.py -c ${config} -o weights=${weights} exclude_nms=True # trt=True

# 5.deploy infer
#CUDA_VISIBLE_DEVICES=0 python deploy/python/infer.py --model_dir=output_inference/${job_name} --image_file=demo/000000014439_640x640.jpg --device=GPU

# 6.deploy speed, add '--run_mode=trt_fp16' to test in TensorRT FP16 mode
#CUDA_VISIBLE_DEVICES=0 python deploy/python/infer.py --model_dir=output_inference/${job_name} --image_file=demo/000000014439_640x640.jpg --device=GPU --run_benchmark=True # --run_mode=trt_fp16

# 7.export onnx
#paddle2onnx --model_dir output_inference/${job_name} --model_filename model.pdmodel --params_filename model.pdiparams --opset_version 12 --save_file ${job_name}.onnx

# 8.onnx speed
#/usr/local/TensorRT-8.0.3.4/bin/trtexec --onnx=${job_name}.onnx --workspace=4096 --avgRuns=10 --shapes=input:1x3x640x640 --fp16
#/usr/local/TensorRT-8.0.3.4/bin/trtexec --onnx=${job_name}.onnx --workspace=4096 --avgRuns=10 --shapes=input:1x3x640x640 --fp32

